<a href="https://colab.research.google.com/github/bwreeves/GoogleColab/blob/main/DirectorySearchOpenSource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello XponentL")

In [ ]:
#install statements
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install pypdf

In [21]:
#import statements
#loader and splitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#pipeline to bring in Flan-T5
from transformers import pipeline
#to import llm object
from langchain.llms import HuggingFacePipeline
#embeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings.base import Embeddings
from InstructorEmbedding import INSTRUCTOR
#vector store
from langchain.vectorstores import Pinecone
import pinecone
#retriever
from langchain.chains import RetrievalQA

In [ ]:
#import the model using a pipeline. I used base becuase my ram crashes eventually if I use bigger. 
model = pipeline(model="google/flan-t5-base", max_size=512, truncation=True)
model.save_pretrained("~/flan-t5-base")

In [ ]:
#connect google drive to import a folder without needing to zip/unzip
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#load directory
path = "/content/gdrive/MyDrive/PdfsForLLM"
loader = PyPDFDirectoryLoader(path)
data = loader.load()

In [ ]:
#split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(data)
print(len(data))

In [ ]:
#create embeddings object
hf = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
ebmeddings = hf

In [34]:
#connect to pinecone and import docs
pinecone.init(
    api_key="920e41ab-b0cb-48f4-9c9b-23208a1fa1bb", environment="us-central1-gcp"
)
index_name = "search-document"
docsearch = Pinecone.from_documents(texts, ebmeddings, index_name=index_name)

In [37]:
#make the llm object
llm = HuggingFacePipeline.from_model_id(model_id="google/flan-t5-base", task="text2text-generation",
                          model_kwargs={"temperature":0})

In [63]:
#make the retrieval, seed it with low token size, include sources
retriever = docsearch.as_retriever(search_kwargs={"k":1})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,
                            return_source_documents=True)

In [ ]:
#question 1 pulls from pdf4
answer = qa("How do you know if a poset is well ordered")
print(answer["result"])
for i in answer["source_documents"]:
  print(i.metadata["source"])
print(answer)

In [ ]:
#question 2 pulls from pd2
answer2 = qa("What does it mean to be relatively prime?")
print(answer2["result"])
for i in answer2["source_documents"]:
  print(i.metadata["source"])

In [ ]:
#question 3 pulls from pdf 5
answer3 = qa("Proof with mathematical induction?")
print(answer3["result"])
for i in answer3["source_documents"]:
  print(i.metadata["source"])